In [1]:
import pandas as pd
import numpy as np
import itertools
import os
from pulp import *

In [2]:
problem = LpProblem("E-Scooter Allocation Profit Maximisation", LpMaximize)

In [3]:
df_distance = pd.read_csv("Data/Model Data - Smaller set Distance.csv")
df_distance.fillna(0, inplace=True)
distance = df_distance.values[:,1:]
distance_dict = { (x,y): distance[x][y] for x in range(distance.shape[0]) for y in range(distance.shape[1])}

In [4]:
locations = list(df_distance['Location'])
location_idx = np.arange(0, len(locations))
loc_count = len(locations)

In [5]:
demand = dict()
files = os.listdir('Data/Reduced set Demand/')
for f in files:
    hour = int(f.split(sep='.', maxsplit=1)[0])
    df = pd.read_csv('Data/Reduced set Demand/' + f)
    demand_arr = df.values[:,1:]
    demand[hour] = demand_arr

In [6]:
demand_dict = dict()
for i,(_,v) in enumerate(demand.items()):
    demand_dict.update({(x,y,i): v[x][y] for x in range(loc_count) for y in range(loc_count)})

In [7]:
# Parameters
M = sys.maxsize
Nmax = 10
Smax = 20
Qmin = 0.35
# Cost values
# cost of maintaining one scooter per kilometer driven
C_scooter_km = 7
C_fixed_scooter = 167
C_fixed_dock = 46
C_fixed_station = 6
# Cost relocation
C_r = 160
# Price rate for kilometer driven
P_km = 0.2
# Price for pickup
P_init = 1

In [8]:
# Penalty
df_penalty = pd.read_csv("Data/Penalty Carbon Costs Smaller set.csv")
C_pen = df_penalty.values[:, 1]

In [9]:
# Sets
T = np.arange(0,len(files))
# each location at a given time
X = list(itertools.product(location_idx, T))
A1 = [(xi, xj) for xi in X for xj in X if xi[0] != xj[0] and xi[1]+1==xj[1]]
A2 = [(xi, xj) for xi in X for xj in X if xi[0]==xj[0] and xi[1]+1==xj[1]]
# Relocation
A3 = [(xi, xj) for xi in X for xj in X if xi[0]!=xj[0] and xi[1]==T[-1] and xj[1]==T[0]]

In [10]:
T

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18])

In [11]:
# Decision Variables
Yi = LpVariable.dicts("Station Presence", location_idx,0,cat=const.LpBinary)
Zi = LpVariable.dicts("Size", location_idx, 0, cat="Continuous")
# Relocation
Rij = LpVariable.dicts("#Relocated_Scooters", A3, 0, cat=const.LpInteger)
Vit = LpVariable.dicts("#Available_Scooters",X,0,cat="Continuous")
Sit = LpVariable.dicts("#Stocked_Scooters", A2, 0, cat="Continuous")
Ditj = LpVariable.dicts("#Used_Scooters", A1, 0 ,cat="Continuous")

In [ ]:
problem+=lpSum(Ditj[((i,ti),(j,tj))]*(distance_dict[(i,j)]*(P_km-C_scooter_km) + P_init) for ((i,ti),(j,tj)) in A1) + \
    lpSum(Rij[((i,ti), (j,tj))]*distance_dict[(i,j)]*C_r for ((i,ti), (j,tj)) in A3) + \
    lpSum(Vit[(i,t)]*C_fixed_scooter for (i,t) in X if t==0) + \
    lpSum(Zi[i]*C_fixed_dock for i in location_idx) + \
    lpSum(Yi[i]*C_fixed_station for i in location_idx), "Objective function"

In [13]:
for (i,ti) in X:
    if ti !=0:
        problem+= Vit[(i,ti-1)] - lpSum(Ditj[((i,ti-1),(j,tj))] for (j,tj) in X if tj==ti and i !=j) + lpSum(Ditj[((j, tj),(i, ti))] for (j,tj) in X if tj == ti-1 and i !=j) == Vit[(i,ti)], f"Availability Balance {(i,ti)}"


In [14]:
for (i,ti) in X:
    if ti!=T[-1]:
        problem+=Vit[(i,ti)] - lpSum(Ditj[((i,ti),(j,tj))]for (j,tj) in X if i !=j and ti+1==tj) == Sit[((i,ti), (i,ti+1))], f"Stocked Scooters for {(i,ti)}"

In [15]:
# Relocation constraint
for i in location_idx:
    problem+=Vit[(i,0)] == Vit[(i,T[-1])] + lpSum(Rij[((j,T[-1]), (i,0))] for j in location_idx if i!=j) - lpSum(Rij[((i,T[-1]), (j,0))] for j in location_idx if i!=j), f"Relocation balance for location {i}"

In [16]:
for i in location_idx:
    problem+=lpSum(Rij[((i,T[-1]), (j,0))] for j in location_idx if i!=j) <= Vit[(i,T[-1])], f"Relocation availability for location {i}"

In [17]:
for (i,ti) in X:
    problem+= Zi[i] >=Vit[(i,ti)], f"Size constraint for {(i,ti)}"

In [18]:
for (i,ti) in X:
    problem+=Vit[(i,ti)] <= M*Yi[i], f"Availability against Dock present{(i,ti)}"

In [19]:
for i in location_idx:
    problem+=Yi[i] <= Zi[i], f"Size constraint for {i}"

In [20]:
for ((i,ti),(j,tj)) in A1:
    problem+=Ditj[((i,ti),(j,tj))]<=demand_dict[(i,j,ti)], f"Maximum Demand for {((i,ti),(j,tj))}"

In [ ]:
problem+=lpSum(v for v in Ditj.values())/lpSum(v for v in demand_dict.values()) >= Qmin, "Minimum satisfied demand"

In [21]:
for i in location_idx:
    problem+=Zi[i] <= Smax, f"Maximum size for {i}"

In [22]:
problem+=lpSum(Yi[i] for i in location_idx) <= Nmax, f"Maximum number of docks"

In [23]:
problem.writeLP("E-scooterProblem.lp")
problem.solve()
LpStatus[problem.status]

'Optimal'

In [24]:
for v in problem.variables():
    if v.varValue>0:
        print(v.name, "=", v.varValue)

#Available_Scooters_(12,_0) = 1.0
#Available_Scooters_(12,_1) = 1.0
#Available_Scooters_(12,_10) = 13.0
#Available_Scooters_(12,_11) = 14.0
#Available_Scooters_(12,_12) = 14.0
#Available_Scooters_(12,_13) = 15.0
#Available_Scooters_(12,_14) = 16.0
#Available_Scooters_(12,_15) = 13.0
#Available_Scooters_(12,_16) = 4.0
#Available_Scooters_(12,_17) = 3.0
#Available_Scooters_(12,_18) = 1.0
#Available_Scooters_(12,_2) = 1.0
#Available_Scooters_(12,_3) = 1.0
#Available_Scooters_(12,_4) = 1.0
#Available_Scooters_(12,_5) = 1.0
#Available_Scooters_(12,_6) = 1.0
#Available_Scooters_(12,_7) = 2.0
#Available_Scooters_(12,_8) = 4.0
#Available_Scooters_(12,_9) = 11.0
#Available_Scooters_(13,_0) = 5.0
#Available_Scooters_(13,_1) = 4.0
#Available_Scooters_(13,_10) = 6.0
#Available_Scooters_(13,_11) = 7.0
#Available_Scooters_(13,_12) = 7.0
#Available_Scooters_(13,_13) = 7.0
#Available_Scooters_(13,_14) = 7.0
#Available_Scooters_(13,_15) = 7.0
#Available_Scooters_(13,_16) = 5.0
#Available_Scooters_(13,_

In [25]:
value(problem.objective)

135463.83435079962

In [26]:
# f = open("obj.txt", "w+")
# print(problem.objective, file=f)
# f.close()

In [27]:
# Extracting the results
#(idx, LpVariable)
size_list = []
for _,v in Zi.items():
    size_list.append(v.value())  

In [28]:
station_list = []
for _,v in Yi.items():
    station_list.append(v.value()) 

In [29]:
df_results = pd.DataFrame(list(zip(locations, station_list, size_list)), columns=["Location","Station", "Size"])
df_results.head()

,Location,Station,Size
0,Balgreen and Roseburn,0.0,0.0
1,Blackhall,0.0,0.0
2,Broughton North and Powderhall,1.0,8.0
3,Broughton South,0.0,0.0
4,Bruntsfield,1.0,3.0


In [30]:
df_results.to_csv("results-reduced.csv", index=False)

In [31]:
for k, v in Sit.items():
    print(str(k[0]) + ', '+ str(k[1]) + "=" + str(v.varValue))

(0, 0), (0, 1)=0.0
(0, 1), (0, 2)=0.0
(0, 2), (0, 3)=0.0
(0, 3), (0, 4)=0.0
(0, 4), (0, 5)=0.0
(0, 5), (0, 6)=0.0
(0, 6), (0, 7)=0.0
(0, 7), (0, 8)=0.0
(0, 8), (0, 9)=0.0
(0, 9), (0, 10)=0.0
(0, 10), (0, 11)=0.0
(0, 11), (0, 12)=0.0
(0, 12), (0, 13)=0.0
(0, 13), (0, 14)=0.0
(0, 14), (0, 15)=0.0
(0, 15), (0, 16)=0.0
(0, 16), (0, 17)=0.0
(0, 17), (0, 18)=0.0
(1, 0), (1, 1)=0.0
(1, 1), (1, 2)=0.0
(1, 2), (1, 3)=0.0
(1, 3), (1, 4)=0.0
(1, 4), (1, 5)=0.0
(1, 5), (1, 6)=0.0
(1, 6), (1, 7)=0.0
(1, 7), (1, 8)=0.0
(1, 8), (1, 9)=0.0
(1, 9), (1, 10)=0.0
(1, 10), (1, 11)=0.0
(1, 11), (1, 12)=0.0
(1, 12), (1, 13)=0.0
(1, 13), (1, 14)=0.0
(1, 14), (1, 15)=0.0
(1, 15), (1, 16)=0.0
(1, 16), (1, 17)=0.0
(1, 17), (1, 18)=0.0
(2, 0), (2, 1)=1.0
(2, 1), (2, 2)=1.0
(2, 2), (2, 3)=1.0
(2, 3), (2, 4)=1.0
(2, 4), (2, 5)=0.0
(2, 5), (2, 6)=0.0
(2, 6), (2, 7)=0.0
(2, 7), (2, 8)=0.0
(2, 8), (2, 9)=3.0
(2, 9), (2, 10)=7.0
(2, 10), (2, 11)=8.0
(2, 11), (2, 12)=8.0
(2, 12), (2, 13)=8.0
(2, 13), (2, 14)=8.0
(2, 14)

In [32]:
total_Sit = dict()
for k,v in Sit.items():
    if k[0][1] not in total_Sit:
        total_Sit[k[0][1]] = int(v.varValue)
    else:
        total_Sit[k[0][1]]+=int(v.varValue)

In [33]:
# Total number of Stocked scooters at a given time
total_Sit

{0: 46,
 1: 56,
 2: 54,
 3: 55,
 4: 22,
 5: 1,
 6: 2,
 7: 1,
 8: 11,
 9: 35,
 10: 50,
 11: 57,
 12: 54,
 13: 53,
 14: 39,
 15: 13,
 16: 4,
 17: 9}

In [34]:
total_Vit = dict()
for k,v in Vit.items():
    if k[1] not in total_Vit:
        total_Vit[k[1]] = int(v.varValue)
    else:
        total_Vit[k[1]]+=int(v.varValue)

In [38]:
print("#Scooters: " + str(total_Vit[0]))

#Scooters: 58


In [40]:
total_demand = 0
for _,v in demand_dict.items():
    total_demand+=v
total_demand

1813

In [43]:
satisfied_demand = 0
for _,v in Ditj.items():
    satisfied_demand+=v.varValue
satisfied_demand

482.0

In [44]:
satisfied_demand/total_demand

0.265857694429123